### Bring Your Own Container: Recommend top 5 books for new users using R

The goal of this notebook is to illustrate how you can train and host R model seamlessly in Amazon SageMaker. In other words, we will through the process of bringing your own docker container to Amazon SageMaker. Rather than reinventing the wheel of training and hosting ML models using SageMaker's built-in algorithms, data scientists and machine learning engineers can re-use their work done in R in SageMaker

In [77]:
import pandas as pd
import boto3
import os
import time
import json
from sagemaker import get_execution_role

### Prepare Dataset
We will first begin by preparing the dataset. The ClndBookRatings.csv is generated from the notebook object2vec_bookratings_reco.ipynb. We will further refine this dataset by only selecting users who have rated greater than 100 books

In [78]:
ip_fn = 'ClndBookRatings.csv' # outliers are removed - remove books with zero ratings
op_fn = 'train_test_bkratings_r.csv' #output file name
bkratings = pd.read_csv(ip_fn)

In [4]:
bkratings.head()

,ISBN,UserID,BookRating,BookTitle,user_ind,book_ind
0,0060160772,275922,10,"Peace, Love and Healing: Bodymind Communicatio...",0,0
1,0060160772,128835,10,"Peace, Love and Healing: Bodymind Communicatio...",1,0
2,0446519138,275922,10,Simple Abundance: A Daybook of Comfort and Joy,0,2576
3,0446519138,267326,9,Simple Abundance: A Daybook of Comfort and Joy,6,2576
4,0446519138,16795,8,Simple Abundance: A Daybook of Comfort and Joy,11,2576


In [14]:
# short list users who have rated at least 100 books
grp_bkratings = bkratings.groupby('user_ind')
fil_bkratings = grp_bkratings.filter(lambda x: x['book_ind'].count() >=100)

In [19]:
fil_bkratings.head()

,ISBN,UserID,BookRating,BookTitle,user_ind,book_ind
1,0060160772,128835,10,"Peace, Love and Healing: Bodymind Communicatio...",1,0
4,0446519138,16795,8,Simple Abundance: A Daybook of Comfort and Joy,11,2576
5,0446519138,11676,8,Simple Abundance: A Daybook of Comfort and Joy,47,2576
6,0446519138,46398,6,Simple Abundance: A Daybook of Comfort and Joy,57,2576
12,0446519138,21014,9,Simple Abundance: A Daybook of Comfort and Joy,745,2576


In [22]:
sel_bkratings = fil_bkratings[['user_ind', 'book_ind', 'BookRating']]
sel_bkratings.to_csv(op_fn, header='true', index=False)

### Setup Permissions to publish docker image to EC2ContainerRegistry (ECR)

In [12]:
bucket = 'ai-in-aws'
prefix = 'sagemaker/byoc-r'
 
# Define IAM role
role = get_execution_role()

In [4]:
role

'arn:aws:iam::109099157774:role/service-role/AmazonSageMaker-ExecutionRole-20180923T113724'

For the SageMakerExecutionRole-20180923T113724 IAM role, you will need both SageMakerFullAccess and AmazonEC2ContainerRegistryFullAccess permissions. Navigate to IAM service, select Roles on the left navigation pane, and search for SageMakerExecutionRole-20180923T113724. And then attach AmazonEC2ContainerRegistryFullAccess policy to the role.

### Publish local docker image to ECR

We will create docker image on the local EC2 instance and then publish it to ECR

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=cosinesimilarity

#Get current account 

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration. Default is us-east-1
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
# Dockerfile is defined in the current directory
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

### Train the Recommender model on user book ratings
We will start my pushing processed user book ratings to s3 bucket

In [15]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', op_fn)).upload_file(op_fn)

In [20]:
region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')

#### Create Training Parameters

In [70]:
r_job = 'BYOC-r' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

print("Training job", r_job)

r_training_params = {
    "RoleArn": role,
    "TrainingJobName": r_job,
    "AlgorithmSpecification": {
        "TrainingImage": '{}.dkr.ecr.{}.amazonaws.com/cosinesimilarity:latest'.format(account, region),
        "TrainingInputMode": "File"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.m4.xlarge",
        "VolumeSizeInGB": 10
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }
    ],
    "OutputDataConfig": {
        "S3OutputPath": "s3://{}/{}/output".format(bucket, prefix)
    },
    "HyperParameters": {
        "method": "Cosine",
        "nn": "10",
        "n_users": "190"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60
    }
}

Training job BYOC-r2019-06-02-22-27-23


#### Create Training Job

In [71]:
%%time

sm = boto3.client('sagemaker')
sm.create_training_job(**r_training_params)

status = sm.describe_training_job(TrainingJobName=r_job)['TrainingJobStatus']
print(status)
sm.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=r_job)
status = sm.describe_training_job(TrainingJobName=r_job)['TrainingJobStatus']
print("Training job ended with status: " + status)
if status == 'Failed':
    message = sm.describe_training_job(TrainingJobName=r_job)['FailureReason']
    print('Training failed with the following error: {}'.format(message))
    raise Exception('Training job failed')

InProgress
Training job ended with status: Completed
CPU times: user 67.4 ms, sys: 0 ns, total: 67.4 ms
Wall time: 4min


#### Create Model
Let's create a model from the training job, pointing to the docker image in ECR and the model artifacts resulting from training job 

In [72]:
r_hosting_container = {
    'Image': '{}.dkr.ecr.{}.amazonaws.com/cosinesimilarity:latest'.format(account, region),
    'ModelDataUrl': sm.describe_training_job(TrainingJobName=r_job)['ModelArtifacts']['S3ModelArtifacts']
}

create_model_response = sm.create_model(
    ModelName=r_job,
    ExecutionRoleArn=role,
    PrimaryContainer=r_hosting_container)

print(create_model_response['ModelArn'])

arn:aws:sagemaker:us-east-1:109099157774:model/byoc-r2019-06-02-22-27-23


In [58]:
r_hosting_container

{'Image': '109099157774.dkr.ecr.us-east-1.amazonaws.com/cosinesimilarity:latest',
 'ModelDataUrl': 's3://ai-in-aws/sagemaker/byoc-r/output/BYOC-r2019-06-02-21-28-51/output/model.tar.gz'}

### Create Model EndPoint
Define the type of infrastructure that needs to be spun up and the model that needs to be hosted on it. 

In [73]:
r_endpoint_config = 'BYOC-r-config-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(r_endpoint_config)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=r_endpoint_config,
    ProductionVariants=[{
        'InstanceType': 'ml.m4.xlarge',
        'InitialInstanceCount': 1,
        'ModelName': r_job,
        'VariantName': 'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

BYOC-r-config-2019-06-02-22-32-11
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:109099157774:endpoint-config/byoc-r-config-2019-06-02-22-32-11


In [74]:
%%time

r_endpoint = 'BYOC-r-endpoint-' + time.strftime("%Y%m%d%H%M", time.gmtime())
print(r_endpoint)
create_endpoint_response = sm.create_endpoint(
    EndpointName=r_endpoint,
    EndpointConfigName=r_endpoint_config)
print(create_endpoint_response['EndpointArn'])

resp = sm.describe_endpoint(EndpointName=r_endpoint)
status = resp['EndpointStatus']
print("Status: " + status)

try:
    sm.get_waiter('endpoint_in_service').wait(EndpointName=r_endpoint)
finally:
    resp = sm.describe_endpoint(EndpointName=r_endpoint)
    status = resp['EndpointStatus']
    print("Arn: " + resp['EndpointArn'])
    print("Status: " + status)

    if status != 'InService':
        raise Exception('Endpoint creation did not succeed')

BYOC-r-endpoint-201906022232
arn:aws:sagemaker:us-east-1:109099157774:endpoint/byoc-r-endpoint-201906022232
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:109099157774:endpoint/byoc-r-endpoint-201906022232
Status: InService
CPU times: user 211 ms, sys: 7.1 ms, total: 218 ms
Wall time: 7min 32s


### Inferences: Invoke Endpoint
Get top 5 recommendations of user 192

In [79]:
ratings = pd.read_csv(op_fn)

runtime = boto3.Session().client('runtime.sagemaker')

payload =  ratings.to_csv(index=False) # get top 5 book recommendations for user 192 (Remember, we trained the model on the first 190 users)
response = runtime.invoke_endpoint(EndpointName='BYOC-r-endpoint-201906022232', #r_endpoint
                                   ContentType='text/csv',
                                   Body=payload)

result = json.loads(response['Body'].read().decode())
result 

[['20445', '2399', '15876', '712', '959']]

In [87]:
# Retrieve book titles from book indexes

bkratings[bkratings.book_ind.isin(result[0])]['BookTitle'].drop_duplicates()

3198                                     The Da Vinci Code
4549                               A Prayer for Owen Meany
19874                                       The Green Mile
31847    The Price of Loyalty: George W. Bush, the Whit...
31882                                     Dark Inheritance
Name: BookTitle, dtype: object